<a href="https://colab.research.google.com/github/s0ye0nyang/DeepLearningZeroToAll/blob/master/assignment10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np

file_data   = "/content/drive/My Drive/Data/mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)
count       = 0     # count for the number of images

#
# normalize the values of the input data to be [0, 1]
#
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

#
# example of distance function between two vectors x and y
#
def distance(x, y):

    d = (x - y) ** 2
    s = np.sum(d)
    # r = np.sqrt(s)

    return(s)

#
# make a matrix each column of which represents an images in a vector form
#
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)

for line in data:

    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    im_vector   = normalize(im_vector)

    list_label[count]       = label
    list_image[:, count]    = im_vector

    count += 1

#
# plot first 150 images out of 10,000 with their labels
#
# f1 = plt.figure(1)

# for i in range(150):

#     label       = list_label[i]
#     im_vector   = list_image[:, i]
#     im_matrix   = im_vector.reshape((size_row, size_col))

#     plt.subplot(10, 15, i+1)
#     plt.title(label)
#     plt.imshow(im_matrix, cmap='Greys', interpolation='None')

#     frame   = plt.gca()
#     frame.axes.get_xaxis().set_visible(False)
#     frame.axes.get_yaxis().set_visible(False)


#plt.show()

#
# plot the average image of all the images for each digit
#
# f2 = plt.figure(2)

# im_average  = np.zeros((size_row * size_col, 10), dtype=float)
# im_count    = np.zeros(10, dtype=int)

# for i in range(num_image):

#     im_average[:, list_label[i]] += list_image[:, i]
#     im_count[list_label[i]] += 1
    

# for i in range(10):

#     im_average[:, i] /= im_count[i]

#     plt.subplot(2, 5, i+1)
#     plt.title(i)
#     plt.imshow(im_average[:,i].reshape((size_row, size_col)), cmap='Greys', interpolation='None')

#     frame   = plt.gca()
#     frame.axes.get_xaxis().set_visible(False)
#     frame.axes.get_yaxis().set_visible(False)

# plt.show()


In [0]:
# learning rate
alpha = 0.4
# 몇 번 연산할 것인지
num_iter = 250
imat = np.eye(10, dtype=int)
lamda = 0.02 # 0.01 -->best!

bias    = np.ones((1,10000), dtype =float)
train = np.vstack((bias[:,:6000], list_image[:,:6000]))
test = np.vstack((bias[:,6000:], list_image[:, 6000:])) 

list_label_onehot = np.zeros((10,10000), dtype=int)
train_label = list_label_onehot[:,:6000]
test_label = list_label_onehot[:,6000:]


for i in range(10000):
  list_label_onehot[:,i] += imat[:,list_label[i]]

def _logistic_unit(_w, node):
  y_ = np.matmul(_w,node)
  return 1.0 / (1 + np.exp(- y_))

def forward(w0, w1, w2, inputx):
  k       = inputx.s
  hape[1]
  y       = np.zeros((196+1,k), dtype=float)
  z       = np.zeros((49+1,k), dtype=float)
  h       = np.zeros((10,k), dtype=float)
  
  for n in range(k):
    y[:,n] = _logistic_unit(w0, inputx[:,n])
    z[:,n] = _logistic_unit(w1, y[:,n])
    h[:,n] = _logistic_unit(w2, z[:,n])
    
  return y,z,h # list_output


def cal_cost(output, onehot):

  _output     = np.asfarray(output)
  oneh        = np.asfarray(onehot)
  cost        = np.zeros((10,output.shape[1]), dtype=float)
  cost        = np.log(_output) * (-oneh) - (1-oneh) * np.log(1 - _output)
     
  return cost.sum() # 수정 전 : axis = 1 [2,3,4,5,5....]


def gradient_descent(w0,w1,w2,y,z,h, _alpha,_lamda):
    
    # x--y--z--h
    gradient0      = np.zeros((196+1,784+1, 6000), dtype=float)
    gradient1      = np.zeros((49+1,196+1, 6000), dtype=float)
    gradient2      = np.zeros((10,49+1, 6000), dtype=float)
    
    _w0      = np.zeros((196+1,784+1), dtype=float)
    _w1      = np.zeros((49+1,196+1), dtype=float)
    _w2      = np.zeros((10,49+1), dtype=float)
    bias    = np.ones((1,10000), dtype =float)

    for u in range(49+1):
      for w in range(10):
        _h = h[w,:]
        _onehot = train_label[w,:]
        gradient2[w,u,:] = ( _onehot - _h ) * z[u,:]
        for v in range(196+1):
          gradient1[u,v,:] += w2[w,u] * gradient2[w,u,:] * y[v,:] *(1-z[u,:])
          
    for k in range(h.shape[1]):
      a = gradient1[:,:,k] * w1
      a = a.sum(axis=0) # 1* 196
      a = a[:,np.newaxis]
      train_ = np.asfarray(train[:,k])[np.newaxis]
      y_ = np.asfarray(1-y[:,k])[:,np.newaxis]


      tmp                  = np.matmul(y_,train_) * np.tile(a,784+1)
      tmp                  = np.asfarray(tmp)
      gradient0[:,:, k]    = tmp
      
    #데이터 총 그래디언트 합을 구한다
    gradient2 = - (1/6000) * gradient2.sum(axis=2) + _lamda * (w2/(784))
    gradient1 = - (1/6000) * gradient1.sum(axis=2) + _lamda * (w1/(196))
    gradient0 = - (1/6000) * gradient0.sum(axis=2) + _lamda * (w0/49)
    

    _w2 = w2 - (gradient2 * alpha)
    _w1 = w1 - (gradient1 * alpha)
    _w0 = w0 - (gradient0 * alpha)
    
    return _w0, _w1, _w2


def cal_accuracy(output, labels):
  
  cnt =  0
  for i in range(len(labels)):
    index = np.argmax(output[:,i])
    if index == labels[i]:
      cnt+=1

  return (cnt/len(labels))*100



def showImage(output, labels):
  correct     =  ["","","","","","","","","",""]
  wrong       =  ["","","","","","","","","",""]
  cnt =  0
  for i in range(len(labels)):
    index = np.argmax(output[:,i])
    if index == labels[i]:
      if correct[index] =="":
        correct[index] = i
    else:
      if wrong[index] == "":
        wrong[index] = i

  return correct, wrong

In [0]:
# train = list_image[:,0:6000]
# test = list_image[:, 6000:-1] 
# train_label = list_label_onehot[:,0:6000]
# test_label = list_label_onehot[:,6000:-1]

def find_optimal(_alpha, _iter, _lamda):
    _cost       =  []
    _cost2      =  []
    _accuracy   =  []
    _accuracy2  =  []
    w =[]
    correct     =  np.zeros(10, dtype=int)
    wrong       =  np.zeros(10, dtype=int)
 

    # 784 196 49 10
    _w0      = np.random.randn(196+1, 784+1)/np.sqrt(784/2)
    _w1      = np.random.randn(49+1, 196+1)/np.sqrt(196/2)
    _w2      = np.random.randn(10, 49+1)/np.sqrt(49/2)
    
    # data seperation and forward propagation 
    
    y,z,h = forward(_w0,_w1,_w2, train)
    ty, tz, th = forward(_w0,_w1,_w2, test)

    for i in range(0,_iter):
      print("iteration:", i)      
      # 업데이트는 잘 됨..
      _w0, _w1, _w2 = gradient_descent(_w0,_w1,_w2,y,z,h,_alpha, _lamda)
      y, z, h       = forward(_w0,_w1,_w2, train)
      ty, tz, th    = forward(_w0,_w1,_w2, test)

      _cost.append(cal_cost(h,train_label))
      _cost2.append(cal_cost(th,test_label))

      _accuracy.append(cal_accuracy(h,list_label[:6000]))
      _accuracy2.append(cal_accuracy(th,list_label[6000:]))
      print(_accuracy[-1])
    
    correct1, wrong1 = showImage(h, list_label[:6000])
    correct2, wrong2 = showImage(th, list_label[6000:])

    return _cost, _cost2, _accuracy, _accuracy2, correct1 ,wrong1, correct2, wrong2

cost, cost2, accuracy, accuracy2, img11, img12, img21, img22 = find_optimal(alpha, num_iter, lamda)


## 1. plot the loss curve 

In [0]:
import matplotlib.pyplot as plt

plt.plot(cost, c ='b')
plt.plot(cost2, c ='r')
plt.show()


## . plot the accuracy value

In [0]:
plt.plot(accuracy, c= 'b')
plt.plot(accuracy2, c= 'r')
plt.show()

## 3. plot the accuracy value

In [0]:
print("training data accuracy : ",accuracy[-1],"%")
print("test data accuracy : ",accuracy2[-1], "%")

## 4. plot the classification example

In [0]:
print("correct answer image : ")
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(list_image[:,img11[i]].reshape((size_row, size_col)), cmap='Greys', interpolation='None')
    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)
plt.show()

print("wrong answer image :")
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(list_image[:,img12[i]].reshape((size_row, size_col)), cmap='Greys', interpolation='None')
    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)
plt.show()


## 5. testing accuracy

In [0]:
print("test data accuracy : ",accuracy2[-1], "%")